In [1]:
import sys
import gzip

import numpy as np
import tensorflow as tf

from binary_sayc import load_deals, DealData
from simulator import Simulator
from eval_bidding import eval_bidding

In [2]:
sess = tf.InteractiveSession()
saver = tf.train.import_meta_graph('../model/sayc-500000.meta')
saver.restore(sess, '../model/sayc-500000')

graph = tf.get_default_graph()

seq_in = graph.get_tensor_by_name('seq_in:0')
seq_out = graph.get_tensor_by_name('seq_out:0')
keep_prob = graph.get_tensor_by_name('keep_prob:0')

out_bid_logit = graph.get_tensor_by_name('out_bid_logit:0')
out_bid_target = graph.get_tensor_by_name('out_bid_target:0')

INFO:tensorflow:Restoring parameters from ../model/sayc-500000


In [3]:
def model(sess, p_keep=1.0):
    def pred_fun(X):
        bids = sess.run(out_bid_logit, feed_dict={seq_in: X, keep_prob: p_keep})
        return sess.run(tf.nn.softmax(bids[-1]))
    return pred_fun

In [4]:
cost = tf.losses.softmax_cross_entropy(out_bid_target, out_bid_logit)

#train_step = tf.train.AdamOptimizer(0.001).minimize(cost)
train_step = graph.get_operation_by_name('softmax_w/Adam')

In [ ]:
reader = load_deals(gzip.open('../deals_bidding_valid_0001.gz'))

In [ ]:
for i, (deal_str, auction_str, contracts) in enumerate(reader):
    if i % 10 == 0:
        sys.stdout.write('.')
    deal_data = DealData.from_deal_auction_string(deal_str, auction_str)
    deal_data.reset_auction()
    init_auction = deal_data.auction
    
    sim = Simulator(deal_data, contracts, model(sess, 0.95))
    
    best_score, best_auction = sim.best_auction(init_auction, 10)
    
    deal_data.auction = best_auction
    
    n_time_steps = (len(best_auction) // 4) + 1
    
    X, y = deal_data.get_binary(n_time_steps)
    
    sess.run(train_step, feed_dict={seq_in: X, seq_out: y, keep_prob: 0.8})

....................................................................................................

In [ ]:
def apply_bidding(bid_model, fin, fout):
    reader = load_deals(fin)
    for i, (deal_str, auction_str, contracts) in enumerate(reader):
        if i % 10 == 0:
            sys.stdout.write('.')
        if i > 100:
            break
        deal_data = DealData.from_deal_auction_string(deal_str, auction_str)
        deal_data.reset_auction()
        init_auction = deal_data.auction
        sim = Simulator(deal_data, contracts, bid_model)
        auction = [('P' if bid == 'PASS' else bid) for bid in sim.simulate_auction(init_auction, True) if bid != 'PAD_START']
        fout.write((' ' + deal_str + '\n').encode('ascii'))
        fout.write((' ' + ' '.join(auction_str.split()[:2] + auction)).encode('ascii'))
        fout.write('\n'.encode('ascii'))
        for contract, (score1, score2) in contracts.items():
            fout.write(('{}\t{}\t{}\n'. format(contract, score1, score2)).encode('ascii'))

In [ ]:
fout = gzip.open('../test1.gz', 'w')
apply_bidding(model(sess), gzip.open('../deals_bidding_valid_0001.gz'), fout)
fout.close()

In [ ]:
eval_bidding(gzip.open('../test1.gz'))